In [158]:
import random
import heapq
from dataclasses import dataclass, field
from typing import List, Tuple, Callable, Any, Set
import numpy as np
from time import time
from tabulate import tabulate
import threading
from collections import deque
from math import ceil
from enum import Enum
from functools import partial
from copy import deepcopy

In [159]:
class LightsOutPuzzle:
    def __init__(self, board: List[List[int]]):
        self.board = np.array(board, dtype=np.uint8)
        self.size = len(board)

    def toggle(self, x, y):
        for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.board[nx, ny] ^= 1

    def is_solved(self):
        return np.all(self.board == 0)

    def get_moves(self):
        return [(x, y) for x in range(self.size) for y in range(self.size)]

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)


In [160]:
def create_random_board(size: int,  seed: int, num_toggles: int = None):
    random.seed(time() if seed is None else seed)

    board = [[0 for _ in range(size)] for _ in range(size)]
    puzzle = LightsOutPuzzle(board)

    if num_toggles is None:
        num_toggles = random.randint(1, size * size)

    for _ in range(num_toggles):
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        puzzle.toggle(x, y)

    return puzzle.board

In [161]:
def show_solution(
    puzzle: LightsOutPuzzle,
    solution: List[Tuple[int, int]],
    algorithm_name: str,
    nodes_visited: int,
    show_steps: bool = False,
):
    print(f"\nSolving with {algorithm_name}:")
    if solution is None:
        print("No solution found.")
        return
    print(f"Solution: {solution}")
    print(f"Nodes visited: {nodes_visited}")
    if show_steps:
        print("\nSolution steps:")
        for i, move in enumerate(solution):
            print(f"\nStep {i+1}: Toggle position {move}")
            puzzle.toggle(*move)
            print(puzzle)
        print(
            "\nFinal state (solved):"
            if puzzle.is_solved()
            else "\nFinal state (not solved):"
        )
        print(puzzle)

In [162]:
tests = [
    create_random_board(3, 42),
    create_random_board(3, 41),
    create_random_board(3, 42, 5),
    create_random_board(4, 42),
    create_random_board(4, 41),
    create_random_board(4, 42, 5),
    #create_random_board(5, 42),
    #create_random_board(5, 41),
   # create_random_board(5, 42, 5),
]

In [163]:
def bfs_solve(puzzle: LightsOutPuzzle):
    
    frontier = deque([(puzzle.board.copy(), [])])
    explored = set() 
    nodes_checked = 0 

    while frontier:
        current_board, path = frontier.popleft()
        nodes_checked += 1 
        
        if np.all(current_board == 0):
            return path, nodes_checked
        
        
        board_tuple = tuple(map(tuple, current_board))
        
        if board_tuple in explored:
            continue
            
        explored.add(board_tuple)
        
        for move in puzzle.get_moves():
            new_board = current_board.copy()
            puzzle_copy = LightsOutPuzzle(new_board)
            puzzle_copy.toggle(move[0], move[1])
            new_path = path + [move]
            
            frontier.append((puzzle_copy.board.copy(), new_path))
    return None, nodes_checked

In [164]:
def ids_solve(puzzle: LightsOutPuzzle):
    
    nodes_checked = [0]
    depth = 0
    
    while True:
        result = depth_limited_search(puzzle, depth, nodes_checked)
        
        if result is not None:
            return result, nodes_checked[0]
        
        depth += 1

In [165]:
def depth_limited_search(puzzle: LightsOutPuzzle, limit: int, nodes_checked: int):

    def iterate_dfs(board, path, depth):
        nodes_checked[0] += 1  
        
        if np.all(board == 0):
            return path
        
        if depth == 0:
            return None
        
        for move in puzzle.get_moves():
            new_board = board.copy()
            puzzle_copy = LightsOutPuzzle(new_board)
            puzzle_copy.toggle(move[0], move[1])
            new_path = path + [move]
            
            result = iterate_dfs(puzzle_copy.board, new_path, depth - 1)
            if result is not None:
                return result
        
        return None
    
    return iterate_dfs(puzzle.board, [], limit)

In [166]:
class SearchStatus(Enum):
    SOLVED = "Solved"
    TIMEOUT = "Timeout"
    NO_SOLUTION = "No Solution"

@dataclass
class SearchResult:
    result: SearchStatus
    solution: Any = None
    nodes_visited: int = None
    time: float = None

    def __str__(self):
        output = f"Result: {self.result.value}\n"
        if self.result == SearchStatus.SOLVED:
            output += f"Solution: {self.solution}\n"
        if self.result != SearchStatus.TIMEOUT:
            output += f"Nodes Visited: {self.nodes_visited}\n"
            output += f"Time Taken: {self.time:.3f} seconds\n"

        return output


DEFAULT_SEARCH_RESULT = SearchResult(SearchStatus.TIMEOUT, None, None, float("inf"))

In [167]:
def run_searches(search_fn: Callable, puzzle: LightsOutPuzzle, name: str, time_limit: int):
    start_time = time()
    
    solution, nodes_checked  = search_fn(puzzle)
    total_time = time() - start_time
    
    return {
        'name': name,
        'solution': solution,
        'time': total_time,
        'nodes_checked': nodes_checked,  
        'steps': len(solution) if solution else 0,
    }


In [171]:
def run_test(test, time_limit):
    puzzle = LightsOutPuzzle(test)
    
  #  print(f"Running test: \n{puzzle}\n")

    bfs_result = run_searches(bfs_solve, deepcopy(puzzle), name="BFS", time_limit=time_limit)
    ids_result = run_searches(ids_solve, deepcopy(puzzle), name="IDS", time_limit=time_limit)
    
    return (
        test,
        bfs_result,
        ids_result,
    )

In [172]:
results = []

for test in tests:
    results.append(run_test(test,180))

In [173]:
table_data = []

for result in results:
    table_data.append(
        [
            "\n".join(" ".join(str(cell) for cell in row) for row in result[0]), 
            f"Solution: {result[1]['solution']}\nSolution Steps: {result[1]['steps']}\nTime: {result[1]['time']:.3f} sec\nNodes Checked: {result[1]['nodes_checked']}",  
            f"Solution: {result[1]['solution']}\nSolution Steps: {result[2]['steps']}\nTime: {result[2]['time']:.3f} sec\nNodes Checked: {result[2]['nodes_checked']}", 
        ]
    )

print("Results:")
print(
    tabulate(
        table_data,
        headers=[
            "Test",
            "BFS", 
            "IDS",  
        ],
        floatfmt=".3f",
        numalign="center",
        stralign="center",
        tablefmt="fancy_grid",
    )
)


Results:
╒═════════╤════════════════════════════════════════════════════╤════════════════════════════════════════════════════╕
│  Test   │                        BFS                         │                        IDS                         │
╞═════════╪════════════════════════════════════════════════════╪════════════════════════════════════════════════════╡
│  1 1 1  │             Solution: [(0, 2), (1, 0)]             │             Solution: [(0, 2), (1, 0)]             │
│  1 1 1  │                 Solution Steps: 2                  │                 Solution Steps: 2                  │
│  1 0 0  │                  Time: 0.009 sec                   │                  Time: 0.000 sec                   │
│         │                 Nodes Checked: 32                  │                 Nodes Checked: 37                  │
├─────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┤
│  1 1 0  │ Solution: [(0, 0), (0, 1), (1, 0), 